### eSIM profiles
- Подготовка файла с eSIM профайлами для выдачи Касмтомерам

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import datetime as dt
from os.path import join

import roamability as rb

In [2]:
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\eSimProfiles\data'

# keys_files = ['esim_rb_5k_18102019.out', 'RB_200804_5K_eSIM.out']
lpa_files = ['esim_rb_5k_18102019.csv', 'RB_200804_5K_eSIM.csv', 'Freecell_200807_5K_eSIM.csv']

df_qr = DataFrame(columns = ['ICCID', 'QR_Code'])

for file in lpa_files:
    df_qr = df_qr.append(pd.read_csv(join(downloads, file), engine='python', dtype='str'))
    
# keys_cols = ['IMSI','PIN1','PUK1','PIN2','PUK2','KIC1','KID1','KIK1']
# df_keys = DataFrame(columns = keys_cols)

# for file in keys_files:
#     df_keys = df_keys.append(pd.read_csv(join(downloads, file), sep=' ', engine='python', dtype='str',
#                                         usecols=keys_cols))

df_qr['ICCID_CD'] = df_qr.ICCID.str.slice(0,19)
df_qr.sort_values(by='ICCID_CD', inplace=True)
df_qr.reset_index(inplace=True, drop=True)

display(df_qr.head(3))

,ICCID,QR_Code,ICCID_CD
0,8997212330099119820,LPA:1$smdp.io$4K-T4R38-P9G6S6,8997212330099119820
1,8997212330099119821,LPA:1$smdp.io$4K-T4R3F-DCQ6YC,8997212330099119821
2,8997212330099119822,LPA:1$smdp.io$4K-T4R3M-101NJ6N,8997212330099119822


In [3]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
,si.ICCID AS ICCID_CD
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
--AND ss.status IN ('Active', 'Disconnected', 'Manually Suspended')
AND
(
si.IMSI BETWEEN '260060145010000' AND '260060145014999' OR
si.IMSI BETWEEN '260060145031000' AND '260060145035999' OR
si.IMSI BETWEEN '260060145026010' AND '260060145030999'
)
'''

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,ICCID_CD
0,Drimsim,Drimsim eSIM,Active,260060145014246,8997219121000091246
1,Drimsim,Drimsim eSIM,Active,260060145014247,8997219121000091247
2,Drimsim,Drimsim eSIM,Active,260060145014248,8997219121000091248


In [4]:
cols = ['ICCID','QR_Code','RESELLER_NAME','ACCOUNT_NAME', 'IMSI', 'SUB_STATUS']
df_batch = pd.merge(df_qr, df_ocs_imsi, on='ICCID_CD', how='left', suffixes=['_S2', '_S1'])

df_batch = df_batch[cols]
df_batch.head()

,ICCID,QR_Code,RESELLER_NAME,ACCOUNT_NAME,IMSI,SUB_STATUS
0,8997212330099119820,LPA:1$smdp.io$4K-T4R38-P9G6S6,Freecell,Freecell_200807_5K_eSIM,260060145026010,Active
1,8997212330099119821,LPA:1$smdp.io$4K-T4R3F-DCQ6YC,Freecell,Freecell_200807_5K_eSIM,260060145026011,Active
2,8997212330099119822,LPA:1$smdp.io$4K-T4R3M-101NJ6N,Freecell,Freecell_200807_5K_eSIM,260060145026012,Active
3,8997212330099119823,LPA:1$smdp.io$4K-T4R3T-1X2K5XY,Freecell,Freecell_200807_5K_eSIM,260060145026013,Active
4,8997212330099119824,LPA:1$smdp.io$4K-T4R47-1HTFLL9,Freecell,Freecell_200807_5K_eSIM,260060145026014,Active


In [5]:
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_batch.IMSI.astype('str').str.slice(0,12)]
df_batch_group = df_batch.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')
df_batch_group.sort_values('IMSI_MIN')

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
46,WMB Limited RT,WMB eSIMs,Active,260060145010,260060145010000,260060145010999,1000
29,STI - Telinta,ESIM_SW_Main_Account,Active,260060145011,260060145011000,260060145011839,839
28,STI - Telinta,ESIM_KALLER,Active,260060145011,260060145011834,260060145011834,1
27,STI - Telinta,ESIM_GlobaleSIM_Website,Active,260060145011,260060145011840,260060145011899,60
25,STI - Telinta,ESIM_Adonis,Active,260060145011,260060145011900,260060145011949,50
26,STI - Telinta,ESIM_GVC,Active,260060145011,260060145011950,260060145011999,50
39,Simtex,Simtex eSIMs - IO,Active,260060145012,260060145012000,260060145012998,813
40,Simtex,eSIM test,Active,260060145012,260060145012557,260060145012557,1
38,Simtex,Mobimatter,Active,260060145012,260060145012558,260060145012736,179
36,Simtex,Arena Club,Active,260060145012,260060145012980,260060145012985,6


In [9]:
df_batch_group.loc[df_batch_group.RESELLER_NAME == 'Roamability Invertory'].sort_values('IMSI_MIN')
df_batch_group.loc[df_batch_group.RESELLER_NAME == 'STI - Telinta'].sort_values('IMSI_MIN')\
# .to_csv(join(downloads, 'Freecell.csv'), index=False)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
28,STI - Telinta,ESIM_SW_Main_Account,Active,260060145011,260060145011000,260060145011839,839
27,STI - Telinta,ESIM_KALLER,Active,260060145011,260060145011834,260060145011834,1
26,STI - Telinta,ESIM_GlobaleSIM_Website,Active,260060145011,260060145011840,260060145011899,60
24,STI - Telinta,ESIM_Adonis,Active,260060145011,260060145011900,260060145011949,50
25,STI - Telinta,ESIM_GVC,Active,260060145011,260060145011950,260060145011999,50
34,STI - Telinta,RB_eSIM_STI_200911_500,Disconnected,260060145014,260060145014251,260060145014251,1
33,STI - Telinta,RB_eSIM_STI_200911_500,Active,260060145014,260060145014252,260060145014750,499
29,STI - Telinta,New SIM cards,Active,260060145031,260060145031000,260060145031999,1000
30,STI - Telinta,New SIM cards,Active,260060145032,260060145032000,260060145032999,1000
31,STI - Telinta,New SIM cards,Active,260060145033,260060145033000,260060145033999,1000


In [10]:
# Prepare the output file

qustomer = 'Moremins'
imsi_start = 260060145035252
given_quantity = 3

current_date = dt.datetime.now().strftime('%y%m%d')

of_name = f'RB_eSIM_{qustomer}_{current_date}_{given_quantity}.csv'
df_customer = df_batch.loc[df_batch.IMSI.isin(range(imsi_start, imsi_start + given_quantity)),
       ['ICCID','QR_Code','IMSI']]

df_customer.QR_Code.replace(to_replace={r'\.*rsp.truphone.com\.*':'smdp.io'}, regex=True, inplace=True)

df_customer.to_csv(join(downloads, of_name), index=False)
df_customer.head()

,ICCID,QR_Code,IMSI
9242,8997212330099156062,LPA:1$smdp.io$4K-U8RS6-7Z2XCG,260060145035252
9243,8997212330099156063,LPA:1$smdp.io$4K-U8RSD-KW0BU9,260060145035253
9244,8997212330099156064,LPA:1$smdp.io$4K-U8RSR-QRVRSB,260060145035254


### Prepare the output file for Customer order

In [5]:
#######################################################################################
# Define variables here
#######################################################################################

esim_out_file_workz = 'Freecell_10_MatchingID.out'

# Format like:
# 89972123300991198102 4K-T4G53-SAL8OK
# 89972123300991198110 4K-T4G5A-GRZD8A
# 89972123300991198128 4K-T4G5H-299K5X

iccid_beg = 8997212330099119810
iccid_end = 8997212330099124809

order_name = 'Freecell_200807_5K_eSIM' # to use as the name of output file for Customer

#######################################################################################

#######################################################################################

In [4]:
# Download the data from the output file from Workz

df_esim = pd.read_csv(join(downloads, esim_out_file_workz), sep=' ', names=['ICCID_C', 'ESIM'])
df_esim['ICCID'] = df_esim.ICCID_C.str.slice(0, 19)
lpa = 'LPA:1$smdp.io$'
df_esim['QR_Code'] = lpa + df_esim.ESIM
df_esim.drop(['ICCID_C', 'ESIM'], axis=1, inplace=True)

df_esim.head()

,ICCID,QR_Code
0,8997212330099119810,LPA:1$smdp.io$4K-T4G53-SAL8OK
1,8997212330099119811,LPA:1$smdp.io$4K-T4G5A-GRZD8A
2,8997212330099119812,LPA:1$smdp.io$4K-T4G5H-299K5X
3,8997212330099119813,LPA:1$smdp.io$4K-T4G5O-1SDCU09
4,8997212330099119814,LPA:1$smdp.io$4K-T4G62-1JISAEX


In [13]:
# Collect subscriber's data from OCS

sql_srt = """
SELECT
a.account_name AS ACCOUNT
,si.ICCID
,spn.phone_number AS MSISDN
,si.IMSI
FROM SUBSCRIBER_IMSIS si
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
LEFT JOIN SUBSCRIBERS s ON si.subscriber_id = s.subscriber_id
LEFT JOIN ACCOUNTS a ON a.account_id = s.account_id
LEFT JOIN SUBSCRIBER_PHONE_NUMBERS spn ON si.subscriber_id = spn.subscriber_id
WHERE ss.STATUS IN ('Active', 'Suspended', 'Pending') AND ss.END_DATE IS NULL
AND si.iccid BETWEEN '{}' AND  '{}'
ORDER BY si.ICCID""".format(iccid_beg, iccid_end)

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

display(df_ocs.groupby('ACCOUNT')['ICCID','IMSI'].agg([('MIN',min),('MAX', max),('NUM','count')]))

df_ocs.head()

ICCID                             \
                               MIN                  MAX   NUM   
ACCOUNT                                                         
New SIM cards  8997212330099151820  8997212330099155809  3990   

                          IMSI                         
                           MIN              MAX   NUM  
ACCOUNT                                                
New SIM cards  260060145031010  260060145034999  3990

,ACCOUNT,ICCID,MSISDN,IMSI
0,New SIM cards,8997212330099151820,487913068010,260060145031010
1,New SIM cards,8997212330099151821,487913068011,260060145031011
2,New SIM cards,8997212330099151822,487913068012,260060145031012
3,New SIM cards,8997212330099151823,487913068013,260060145031013
4,New SIM cards,8997212330099151824,487913068014,260060145031014


In [24]:
# Megre DataFrames and save to file

pd.merge(df_ocs, df_esim, how='outer', on='ICCID')\
.to_csv(join(downloads, f'{order_name}.csv'), index=False)

### Developments

In [22]:
order_name = 'STI' # to use as the name of output file for Customer
input_file = 'RB_eSIM_STI_201013_3990.csv'

iccid_beg = 8997212330099151820
iccid_end = 8997212330099155809

sql_srt = """
SELECT
a.account_name AS ACCOUNT
,si.ICCID
,spn.phone_number AS MSISDN
,si.IMSI
FROM SUBSCRIBER_IMSIS si
LEFT JOIN SUBSCRIBER_STATUS ss ON si.subscriber_id = ss.subscriber_id
LEFT JOIN SUBSCRIBERS s ON si.subscriber_id = s.subscriber_id
LEFT JOIN ACCOUNTS a ON a.account_id = s.account_id
LEFT JOIN SUBSCRIBER_PHONE_NUMBERS spn ON si.subscriber_id = spn.subscriber_id
WHERE ss.STATUS IN ('Active', 'Suspended', 'Pending') AND ss.END_DATE IS NULL
AND si.iccid BETWEEN '{}' AND  '{}'
ORDER BY si.ICCID""".format(iccid_beg, iccid_end)

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_ocs = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

display(df_ocs.groupby('ACCOUNT')['ICCID','IMSI'].agg([('MIN',min),('MAX', max),('NUM','count')]))

df_ocs.head()

ICCID                             \
                               MIN                  MAX   NUM   
ACCOUNT                                                         
New SIM cards  8997212330099151820  8997212330099155809  3990   

                          IMSI                         
                           MIN              MAX   NUM  
ACCOUNT                                                
New SIM cards  260060145031010  260060145034999  3990

,ACCOUNT,ICCID,MSISDN,IMSI
0,New SIM cards,8997212330099151820,487913068010,260060145031010
1,New SIM cards,8997212330099151821,487913068011,260060145031011
2,New SIM cards,8997212330099151822,487913068012,260060145031012
3,New SIM cards,8997212330099151823,487913068013,260060145031013
4,New SIM cards,8997212330099151824,487913068014,260060145031014


In [23]:
df_esim = pd.read_csv(join(downloads, input_file), dtype='str')
df_esim.head()

,ICCID,QR_Code,IMSI
0,8997212330099151820,LPA:1$smdp.io$4K-U7ZG4-173EZP4,260060145031010
1,8997212330099151821,LPA:1$smdp.io$4K-U7ZGB-12KUH9T,260060145031011
2,8997212330099151822,LPA:1$smdp.io$4K-U7ZGP-98B7WO,260060145031012
3,8997212330099151823,LPA:1$smdp.io$4K-U7ZGW-U1TSO7,260060145031013
4,8997212330099151824,LPA:1$smdp.io$4K-U7ZH3-9FU0WY,260060145031014


In [24]:
# Megre DataFrames and save to file

pd.merge(df_ocs, df_esim, how='outer', on=['ICCID','IMSI'])\
.to_csv(join(downloads, f'{order_name}.csv'), index=False)